In [1]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import PIL

In [2]:
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from tensorflow.keras.layers import Reshape 

In [3]:
test = r'C:\Users\sonuc\Desktop\Data_Science\Dog and cat - Data_Augmentation\test'
train = r'C:\Users\sonuc\Desktop\Data_Science\Dog and cat - Data_Augmentation\train'

In [4]:
tra = VGG16()

In [5]:
tra.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

As we know from above Vgg16 is functional model and to cut the model last layer and than 

adding its whole layer to our last layer we need to convert it to Sequential model

Checking and cutting Vgg16 last layer by running for loop

In [6]:
for layer in tra.layers[:-1]:
    print(layer.__class__.__name__)

InputLayer
Conv2D
Conv2D
MaxPooling2D
Conv2D
Conv2D
MaxPooling2D
Conv2D
Conv2D
Conv2D
MaxPooling2D
Conv2D
Conv2D
Conv2D
MaxPooling2D
Conv2D
Conv2D
Conv2D
MaxPooling2D
Flatten
Dense
Dense


now we can see last layer removed

now we removed last layer so we add this to our sequential layer for ladding our last layer

same as we added earlier in our sequential layer but this we add ready made layer with our last layer made by us

In [7]:
model = Sequential()

In [8]:
for layer in tra.layers[:-1]:
    model.add(layer)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

now we can see last layer removed


now we will freeze our layers because we don't want to change the weight durning backpropagation of our reday made layer(vgg16) we added

In [10]:
# freezing all layers
for layer in model.layers:
    layer.trainable = False

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

As we can now see there is no trainable parameters in model summary

Adding last dense layer with only one node

In [12]:
model.add(Dense(1, activation ='sigmoid'))

Afer adding one node dense layer we again check model summary and than we train the trainable parameter

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [15]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train,  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        test,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

Found 4000 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.


In [16]:
model.fit(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save('first_try.h5')  # always save your weights after training or during training

Epoch 1/50
125/125 [==============================] - 34s 216ms/step - loss: 0.5513 - accuracy: 0.7105 - val_loss: 0.3887 - val_accuracy: 0.8650
Epoch 2/50
125/125 [==============================] - 27s 213ms/step - loss: 0.4415 - accuracy: 0.7930 - val_loss: 0.5739 - val_accuracy: 0.6575
Epoch 3/50
125/125 [==============================] - 27s 215ms/step - loss: 0.3658 - accuracy: 0.8345 - val_loss: 0.3282 - val_accuracy: 0.8575
Epoch 4/50
125/125 [==============================] - 27s 219ms/step - loss: 0.3559 - accuracy: 0.8485 - val_loss: 0.2746 - val_accuracy: 0.9000
Epoch 5/50
125/125 [==============================] - 27s 218ms/step - loss: 0.3314 - accuracy: 0.8555 - val_loss: 0.2732 - val_accuracy: 0.8900
Epoch 6/50
125/125 [==============================] - 27s 217ms/step - loss: 0.3269 - accuracy: 0.8630 - val_loss: 0.2593 - val_accuracy: 0.8938
Epoch 7/50
125/125 [==============================] - 27s 218ms/step - loss: 0.3085 - accuracy: 0.8740 - val_loss: 0.3688 - val_ac